This notebook can be used to run the simulation instead of having to type the commands into the command line interface

In [1]:
# Copyright (c) 2022 The MATCH Authors. All rights reserved.
# Licensed under the GNU AFFERO GENERAL PUBLIC LICENSE Version 3 (or later), which is in the LICENSE file.

import multiprocessing as mp
import os, sys
sys.path.append(r'C:\Users\jiao\Desktop\TIoCS\MATCH-model-master\MATCH-model\match_model')
import generate_input_files
import shutil
import time
from pathlib import Path

########################################################################
model_run_location = r"C:\Users\jiao\Desktop\TIoCS\MATCH-model-master\MATCH-model\MODEL_RUNS\example"
########################################################################

# ensure that the filepath string uses forward slashes rather than backslashes
model_run_location = model_run_location.replace("\\", "/")

# if the location is a relative filepath
if "../" in model_run_location:
    model_workspace = Path.cwd() / model_run_location
# otherwise if it is an absoluate filepaht
elif ":/" in model_run_location:
    model_workspace = Path(model_run_location)


# 提高效率的 if...else...，對結果沒有影響，因為在執行 generate_input_files.generate_inputs 後會把原始資料存到某些模型吃到的位置，若執行過不用再抓一次原始資料
# check if the directory exists
if os.path.exists(model_workspace / "inputs"):
    # check that an inputs version file exists
    if os.path.exists(model_workspace / "inputs_version.txt"):
        # get the version number
        with open(model_workspace / "inputs_version.txt", "r") as i:
            inputs_version = i.read()
        # get the current version number
        version_path = Path.cwd() / "version.py"
        version = {}
        with open(version_path) as f:
            exec(f.read(), version)
        version = version["__version__"]
        # check if the versions match
        if version == inputs_version:
            print(
                f"Input files already generated with current software version ({version})"
            )
        else:
            print(
                f"Inputs were generated using version {inputs_version}. Current version is {version}"
            )
            print(
                "Removing old files and re-generating inputs with current version. Please wait..."
            )
            old_files = os.listdir(model_workspace)
            old_files.remove("model_inputs.xlsx")
            for f in old_files:
                try:
                    os.remove(model_workspace / f)
                except PermissionError:
                    shutil.rmtree(model_workspace / f)
            generate_input_files.generate_inputs(model_workspace)
    else:
        generate_input_files.generate_inputs(model_workspace)
# if the directory does not exist, generate the input files
else:
    print("Generating inputs now...")
    generate_input_files.generate_inputs(model_workspace)

Setting up model directory...
Copying CBC solver to model run directory...
Loading data from model_inputs.xlsx


C:\Users\jiao\Desktop\TIoCS\MATCH-model-master\MATCH-model\match_model\generate_input_files.py:537: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.9801495 0.995    ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  xl_gen.loc[xl_gen["gen_tech"] == "Solar_PV", "solar_age_degredation"] = (
C:\Users\jiao\Desktop\TIoCS\MATCH-model-master\MATCH-model\match_model\generate_input_files.py:304: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  levelized_lrmers = pd.concat([levelized_lrmers, region_data], axis=0)


Generating capacity factor timeseries for set_1
{'Resource': {'resource_height': 100}, 'Turbine': {'wind_resource_shear': 0.15, 'wind_turbine_hub_ht': 94}, 'Powercurve': {'turbine_size': 3450, 'rotor_diameter': 112, 'max_cp': 0.45, 'max_tip_speed': 80, 'max_tip_sp_ratio': 8, 'cut_in': 3, 'cut_out': 25, 'drive_train': 0}, 'Losses': {'avail_bop_loss': 0, 'avail_grid_loss': 0, 'avail_turb_loss': 0, 'ops_env_loss': 0, 'ops_grid_loss': 0, 'ops_load_loss': 0, 'ops_strategies_loss': 0}, 'Farm': {'system_capacity': 3450, 'wind_farm_xCoordinates': [0], 'wind_farm_yCoordinates': [0]}}
Year,Month,Day,Hour,Minute,wind speed at 100m (m/s),wind direction at 100m (deg),air temperature at 100m (C),air pressure at 100m (Pa)
['2012', '1', '1', '0', '30', '7.48', '36.88', '15.370000000000001', '89390']
['2012', '1', '1', '2', '30', '8.45', '36.03', '15.43', '89380']
{'heights': [2012.0, 1.0, 1.0, 2.0, 30.0, 8.45, 36.03, 15.43, 89380.0], 'fields': []}
['2012', '1', '1', '0', '30', '7.48', '36.88', '15.370

ValueError: 2012 required for wind data

# Solve Model Scenarios

### Option 1: Run as many parallel threads as possible

If you would like to run as many scenarios in parallel as your machine can support, run the following cell. 

This will identify how many processor cores (n) exist on your machine, and automatically run (n-1) scenarios (or the total number of scnearios, whichever is less). This should leave at least one core of your machine open for other tasks, but we still recommend that you close all other windows/processes before starting this option.

In [ ]:
num_processors = mp.cpu_count()
print(f"This machine has {num_processors} CPU cores")
# get the number of scenarios to run
num_scenarios = len(os.listdir(model_workspace / "inputs"))
print(f"There are {num_scenarios} scenarios to run")
# determine how many threads to open
parallel_threads = min(num_processors - 1, num_scenarios)
print(f"Running {parallel_threads} scenarios in parallel...")
i = 0
while i < parallel_threads:
    os.system(
        f'start cmd /k "cd {model_run_location} & activate match_model & match solve-scenarios"'
    )
    time.sleep(2)
    i += 1

## Option 2: Start scenarios one at a time

If you would like to only open a single solver window, run the following cell. Each time you run the following cell will open a new parallel solver.

In [3]:
os.system(
    f'start cmd /k "cd {model_run_location} & activate match_model & match solve-scenarios"'
)

0